In [367]:
# default_exp metaflow

In [368]:
# export


import asyncio
import multiprocessing
import os
import subprocess
from pathlib import Path, PosixPath
from typing import Any, Dict, Iterable
from itertools import product

from fastcore.script import call_parse
from nbdev.export import Config, find_default_export, nbglob, read_nb
from sciflow.data_handler import extract_param_meta
from sciflow.params import params_as_dict
from sciflow.parse_module import FuncDetails, extract_steps

# Sciflow Notebook to MetaFlow Flow

> Converts from a `sciflow` format notebook to a `metaflow` flow. 

Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [369]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [370]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")

In [371]:
# export


def titleize(name):
    return name.title().replace("_", "")

In [372]:
assert titleize("snake_case") == "SnakeCase"

In [373]:
# export


def rename_steps_for_metaflow(steps):
    for i, step in enumerate(steps):
        if i == 0:
            step.name = "start"

In [374]:
steps = extract_steps(test_module)

In [375]:
no_steps = extract_steps(os.path.join(Config().path("lib_path"), f"_nbdev.py"))
assert len(no_steps) == 0

In [376]:
assert ["first", "preprocess", "train", "last"] == [step.name for step in steps]
rename_steps_for_metaflow(steps)
assert ["start", "preprocess", "train", "last"] == [step.name for step in steps]

In [377]:
# export


def indent_multiline(multiline_text, indent=1):
    lines = multiline_text.strip().split("\n")
    spaces = "".join(["    " for _ in range(indent)])
    for i in range(len(lines)):
        prefix = spaces if i > 0 else spaces + '"""'
        lines[i] = prefix + lines[i]
    return "\n".join(lines) + '"""'

In [378]:
text = """
Some text
:param param: text
"""
assert '    """Some text\n    :param param: text"""' == indent_multiline(text)

In [379]:
# Test Returns

In [380]:
nb_path = Path(os.path.join("test", "test_clustering.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")
steps = extract_steps(test_module)

In [381]:
# export


def nb_to_metaflow(nb_path: Path, flow_path: Path, silent=True, track_experiment=True):
    nb = read_nb(nb_path)
    lib_name = Config().lib_name
    module_name = find_default_export(nb["cells"])
    if not module_name:
        return
    module_name = module_name
    path_sep_module_name = module_name.replace(".", "/")
    nb_name = os.path.basename(nb_path)
    exported_module = os.path.join(
        Config().path("lib_path"), f"{path_sep_module_name}.py"
    )
    steps = extract_steps(exported_module)
    if len(steps) == 0:
        return
    orig_step_names = [step.name for step in steps]
    if len(steps) == 1:
        steps.append(FuncDetails("end", None, None, False, "", "pass"))
    params = params_as_dict(nb_path)
    if len(params) == 0:
        print(f"No params cell found for: {os.path.basename(nb_path)}")
    flow_class_name = f"{titleize(extract_module_only(module_name))}Flow"
    rename_steps_for_metaflow(steps)
    write_module_to_file(
        flow_path,
        flow_class_name,
        lib_name,
        module_name,
        orig_step_names,
        steps,
        params,
        track_experiment,
    )
    if not silent:
        print(
            f"Converted {nb_name} to {flow_class_name} in: {os.path.basename(flow_path)}"
        )

In [382]:
pkg_module_name = "test.module"
module_name = "module"
path_sep_module_name = module_name.replace(".", "/")

In [383]:
# export


def extract_module_only(package_module_name):
    module_name = package_module_name
    if "." in module_name:
        package_name, module_name = module_name.split(".")
    return module_name

In [384]:
assert "module" == extract_module_only(module_name)
assert "module" == extract_module_only(pkg_module_name)

In [385]:
#  export


def write_module_to_file(
    flow_path: Path,
    flow_class_name: str,
    lib_name: str,
    module_name: str,
    orig_step_names: Iterable[str],
    steps: Iterable[FuncDetails],
    params: dict,
    track_experiment: bool,
):
    if not os.path.exists(flow_path.parent):
        os.mkdir(flow_path.parent)
    fq_module_name = f"{lib_name}.{module_name}"
    param_meta = extract_param_meta(fq_module_name, params)
    with open(flow_path, "w") as flow_file:
        flow_file.write("#!/usr/bin/env python\n")
        flow_file.write("# coding=utf-8\n")
        flow_file.write("# SCIFLOW GENERATED FILE - EDIT COMPANION NOTEBOOK\n")
        has_mf_param = any((p.has_metaflow_param for p in param_meta.values()))
        has_json_param = any((p.is_json_type for p in param_meta.values()))
        mf_params_import = "from metaflow import FlowSpec, step, current"
        if has_mf_param:
            mf_params_import += ", Parameter"
        if has_json_param:
            mf_params_import += ", JSONType"
            flow_file.write("import json\n")
        flow_file.write(mf_params_import + "\n")
        flow_file.write(f"from {fq_module_name} import {', '.join(orig_step_names)}\n")
        if len(params) > 0:
            flow_file.write(
                f"from {fq_module_name} import {', '.join(params.keys())}\n"
            )

        if track_experiment:
            flow_file.write("from sacred import Experiment\n")
            flow_file.write("from sciflow.lake_observer import AWSLakeObserver\n")
            flow_file.write("import time")
            write_observers(flow_file, module_name, Config().bucket, Config().lib_name)

        flow_file.write(f"\n\nclass {flow_class_name}(FlowSpec):\n")
        single_indent = "    "
        write_params(flow_file, param_meta, single_indent)
        flow_file.write(f"{single_indent}artifacts = []\n")
        flow_file.write(f"{single_indent}metrics = []\n")
        flow_file.write("\n")
        write_steps(flow_file, steps, orig_step_names, param_meta, single_indent)
        write_track_flow(flow_file, track_experiment)
        flow_file.write("\n")

        flow_file.write('if __name__ == "__main__":\n')
        flow_file.write(f"{single_indent}{flow_class_name}()")

In [386]:
# export


def write_observers(flow_file, module_name, bucket_name, project):
    experiment_name = extract_module_only(module_name)
    sacred_setup = f"""

ex = Experiment("{experiment_name}")
# TODO inject observers
obs = AWSLakeObserver(experiment_name="{experiment_name}")
ex.observers.append(obs)

@ex.config
def config():
    flow_run_id = None
    artifacts = []
    metrics = []
    """
    flow_file.write(sacred_setup)

In [387]:
# export


def write_track_flow(flow_file, track_experiment):
    track_flow = """
    @step
    def end(self):
        flow_info = {
            "flow_name": current.flow_name,
            "run id": current.run_id,
            "origin run id": current.origin_run_id,
            "pathspec": current.pathspec,
            "namespace": current.namespace,
            "username": current.username,
            "flow parameters": str(current.parameter_names),
            "run_time_mins": round((time.time() - self.__getattr__('start_time')) / 60.0, 1)
        }
    
        run = ex.run(config_updates={'artifacts': self.__getattr__('artifacts'),
                                    'metrics': self.__getattr__('metrics')},
                     meta_info = flow_info)
        
    @ex.main
    def track_flow(artifacts, metrics, _run):
        for artifact in artifacts:
            _run.add_artifact(artifact)
        for metric_name, metric_value, step in metrics:
            _run.log_scalar(metric_name, metric_value, step)
    """
    if not track_experiment:
        track_flow = """
    @step
    def end(self):
        pass
    """
    flow_file.write(track_flow)

In [388]:
#  export


def write_params(flow_file, param_meta, single_indent):
    for param in param_meta.keys():
        if param_meta[param].is_scalar:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default={param})\n"
            )
        elif param_meta[param].is_json_type:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=json.dumps({param}), type=JSONType)\n"
            )
        elif param_meta[param].instance_type == PosixPath:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=str({param}))\n"
            )

In [389]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_data_handling.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_data_handling", params)

In [390]:
assert any((p.has_metaflow_param for p in param_meta.values()))
assert any((p.is_json_type for p in param_meta.values()))

In [391]:
# export


def format_arg(arg, param_meta):
    if arg in param_meta and not param_meta[arg].has_metaflow_param:
        result = arg
    else:
        result = "self." + arg
    return result


def write_steps(flow_file, steps, orig_step_names, param_meta, single_indent):
    for i, step in enumerate(steps):
        flow_file.write(f"{single_indent}@step\n")
        flow_file.write(f"{single_indent}def {step.name}(self):\n")
        if step.docstring:
            flow_file.write(f"{indent_multiline(step.docstring, 2)}\n")
        # Check for padded step
        if i < len(orig_step_names):
            flow_step_args = ""
            if len(step.args) > 0:
                flow_step_args = ", ".join(
                    [format_arg(a, param_meta) for a in step.args.split(",")]
                )
            if not step.has_return:
                flow_file.write(
                    f"{single_indent}{single_indent}{orig_step_names[i]}({flow_step_args})\n"
                )
            else:
                if step.return_stmt in param_meta:
                    raise ValueError(
                        f"[{os.path.basename(flow_file.name)}] step return variable {step.return_stmt} shadows a parameter name - parameters must be unique"
                    )
                flow_file.write(
                    f"{single_indent}{single_indent}results = {orig_step_names[i]}({flow_step_args})\n"
                )
                write_track_capture(flow_file)
        if i == 0:
            flow_file.write(
                f"{single_indent}{single_indent}self.start_time = time.time()\n"
            )
        if i < len(steps):
            next_step = "end" if i == len(steps) - 1 else steps[i + 1].name
            flow_file.write(
                f"{single_indent}{single_indent}self.next(self.{next_step})\n"
            )
        flow_file.write("\n")

In [392]:
# export


def write_track_capture(flow_file):
    flow_file.write(
        f"""
        for key in results.keys():
            if key in self.__dict__:
                self.__dict__[key] = self.__dict__[key] + results[key]
            else:
                self.__dict__[key] = results[key]

"""
    )

In [393]:
# export


def get_module_name(nb_path):
    nb = read_nb(nb_path)
    module_name = find_default_export(nb["cells"])
    return module_name

# Test Flow Generation

In [394]:
def get_flow_path(nb_path):
    return Path(
        os.path.join(
            Config().path("flows_path"),
            f"{get_module_name(nb_path).split('.')[-1]}.py",
        )
    )

In [395]:
assert Path(
    "/home/jovyan/git/sciflow/nbs/test/flows/test_data_handling.py"
) == get_flow_path(nb_path)

In [396]:
flow_path = Path(
    os.path.join(
        Path(".").resolve(),
        "test",
        "flows",
        f"{get_module_name(nb_path).split('.')[-1]}.py",
    )
)

In [397]:
assert (
    Path(os.path.join(Path(".").resolve(), "test", "flows", "test_data_handling.py",))
    == flow_path
)

In [398]:
nb_to_metaflow(nb_path, flow_path, silent=False, track_experiment=True)

Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [399]:
nb_to_metaflow(
    Path("/home/jovyan/git/sciflow/nbs/test/test_clustering.ipynb"),
    Path("/home/jovyan/git/sciflow/nbs/test/flows/test_clustering.py"),
    silent=False,
    track_experiment=True,
)

Converted test_clustering.ipynb to TestClusteringFlow in: test_clustering.py


# Ignore notebooks without Sciflow steps

In [400]:
nb_to_metaflow("packaging.ipynb", flow_path, silent=False)

In [401]:
# export


def generate_flows(config: Config):
    flows_dir = config.path("flows_path")
    nb_paths = nbglob(recursive=True)
    for nb_path in nb_paths:
        flow_module_name = os.path.basename(nb_path).replace("ipynb", "py")
        nb_to_metaflow(
            nb_path, Path(os.path.join(flows_dir, flow_module_name)), silent=False
        )

In [402]:
generate_flows(Config(cfg_name="test/settings.ini"))

No params cell found for: test_clustering_no_params.ipynb
Converted test_clustering_no_params.ipynb to TestClusteringNoParamsFlow in: test_clustering_no_params.py
Converted test_clustering.ipynb to TestClusteringFlow in: test_clustering.py
Converted test_export.ipynb to TestExportFlow in: test_export.py
Converted test_module.ipynb to TestModuleFlow in: test_module.py
Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [403]:
# export


@call_parse
def sciflow_generate():
    generate_flows(Config())

# Verify Flow Generation

> Check that flows are valid and that they run. Running is a longer operation so be careful not to overload your build with slow cycles.

In [404]:
# export


def check_flows(config, flow_command="show", ignore_suffix="_no_params.py"):
    flow_results = {}
    flows_dir = config.path("flows_path")
    if ignore_suffix:
        flow_paths = [p for p in os.listdir(flows_dir) if not p.endswith(ignore_suffix)]
    else:
        flow_paths = os.listdir(flows_dir)
    for flow_path in flow_paths:
        flow_name = os.path.basename(flow_path)
        if flow_path.endswith(".py"):
            ret_code, output = check_flow(
                flows_dir, flow_path, flow_command=flow_command
            )
            flow_results[flow_name] = ret_code, output
            if ret_code == 0:
                print(f"Flow: {flow_name} {flow_command} verified")
            else:
                print(
                    f"Flow: {flow_name} {flow_command} verification failed\nDetails:\n{output}"
                )

In [405]:
# export


def prep_mf_env():
    if "USER" not in os.environ:
        try:
            os.environ["USER"] = os.environ["GIT_COMMITTER_NAME"]
        except KeyError:
            raise EnvironmentError(
                "Metaflow requires a known user for tracked execution. Add USER or GIT_COMMITTER_NAME to Jupyter environment variables"
            )

In [406]:
# export


def run_shell_cmd(script):
    pipe = subprocess.Popen(
        "%s" % script, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True
    )
    output = pipe.communicate()[0]
    return pipe, output.decode("utf-8").strip()

In [407]:
# export


def check_flow(flows_dir, flow_module, flow_command="show", params=None):
    prep_mf_env()
    if params:
        args = " ".join([f"--{p[0]} {p[1]}" for p in params])
        script = (
            f"python '{os.path.join(flows_dir, flow_module)}' {flow_command} {args}"
        )
    else:
        script = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command}"
    pipe, output = run_shell_cmd(script)
    return pipe.returncode, output

In [408]:
params = [("speed", "fast"), ("traffic_percent", 10)]
assert "--speed fast --traffic_percent 10" == " ".join(
    [f"--{p[0]} {p[1]}" for p in params]
)

In [409]:
check_flows(Config(cfg_name="test/settings.ini"), ignore_suffix=None)

Flow: test_module.py show verified
Flow: test_clustering_no_params.py show verified
Flow: test_clustering.py show verified
Flow: test_data_handling.py show verified
Flow: test_export.py show verified


In [410]:
# slow

check_flows(Config(cfg_name="test/settings.ini"), "--no-pylint run")

Flow: test_module.py --no-pylint run verification failed
Details:
Metaflow 2.2.7 executing TestModuleFlow for user:e02079
Validating your flow...
    The graph looks good!
2021-04-22 13:54:47.250 Workflow starting (run-id 1619099687246855):
2021-04-22 13:54:47.255 [1619099687246855/start/1 (pid 6974)] Task is starting.
2021-04-22 13:54:48.245 [1619099687246855/start/1 (pid 6974)] 2
2021-04-22 13:54:48.402 [1619099687246855/start/1 (pid 6974)] Task finished successfully.
2021-04-22 13:54:48.408 [1619099687246855/end/2 (pid 6986)] Task is starting.
2021-04-22 13:54:49.409 [1619099687246855/end/2 (pid 6986)] INFO - test_module - Running command 'track_flow'
2021-04-22 13:54:49.412 [1619099687246855/end/2 (pid 6986)] ERROR - test_module - Failed after 0:00:00!
2021-04-22 13:54:49.413 [1619099687246855/end/2 (pid 6986)] ERROR - test_module - Traceback (most recent call last):
2021-04-22 13:54:49.416 [1619099687246855/end/2 (pid 6986)]   File "/home/jovyan/envs/discovery/lib/python3.7/site-p

In [411]:
# export


def run_flow(nb_path, params=None):
    flow_path = get_flow_path(nb_path)
    print(f"Running flow: {os.path.basename(flow_path)}")
    ret_code, output = check_flow(
        flow_path.parent,
        os.path.basename(flow_path),
        flow_command="--no-pylint run",
        params=params,
    )
    return ret_code, output

In [412]:
# slow

ret_code, output = run_flow(
    Path("/home/jovyan/git/sciflow/nbs/test/test_clustering.ipynb"),
    params=[("traffic_percent", 10), ("speed", "fast")],
)
print(output)
assert ret_code == 0

Running flow: test_clustering.py
Metaflow 2.2.7 executing TestClusteringFlow for user:e02079
Validating your flow...
    The graph looks good!
2021-04-22 13:56:37.651 Workflow starting (run-id 1619099797648348):
2021-04-22 13:56:37.658 [1619099797648348/start/1 (pid 7281)] Task is starting.
2021-04-22 13:56:39.883 [1619099797648348/start/1 (pid 7281)] Task finished successfully.
2021-04-22 13:56:39.890 [1619099797648348/preprocess/2 (pid 7296)] Task is starting.
2021-04-22 13:57:08.522 [1619099797648348/preprocess/2 (pid 7296)] Task finished successfully.
2021-04-22 13:57:08.529 [1619099797648348/fit/3 (pid 7329)] Task is starting.
2021-04-22 13:57:10.739 [1619099797648348/fit/3 (pid 7329)] Task finished successfully.
2021-04-22 13:57:10.746 [1619099797648348/evaluate/4 (pid 7344)] Task is starting.
2021-04-22 13:57:12.954 [1619099797648348/evaluate/4 (pid 7344)] Task finished successfully.
2021-04-22 13:57:12.961 [1619099797648348/end/5 (pid 7359)] Task is starting.
2021-04-22 13:57:1

AssertionError: 

# Aynsc Flow Running

> Run the flow you are working on from the notebook you are working on. This maximises the amount fo experiments you can run as you don't have down time. While long running tasks are running you can keep exploring! :-)

In [ ]:
# export


async def run_flow_task(flow_path, param_grid=None):
    experiment_id = None
    flows_dir = flow_path.parent
    flow_module = os.path.basename(flow_path)
    flow_command = "--no-pylint run"
    prep_mf_env()
    if params:
        args = " ".join([f"--{k} {v}" for k, v in param_grid.items()])
        cmd = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command} {args}"
    else:
        cmd = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command}"
    proc = await asyncio.create_subprocess_shell(
        cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )

    stdout, stderr = await proc.communicate()

    print(f"[{cmd!r} exited with {proc.returncode}]")
    if stdout:
        output = stdout.decode("utf-8").strip()
        experiment_id = [
            s.strip('"')
            for s in re.search('Started run with ID "\d+"', output)
            .group(0)
            .split("ID ")
        ][1]
        print(f"[stdout]\n{output}")
    if stderr:
        print(f'[stderr]\n{stderr.decode("utf-8").strip()}')

    return experiment_id

In [ ]:
# export


def run_flow_async(nb_path, params=None):
    flow_path = get_flow_path(nb_path)
    loop = asyncio.get_event_loop()
    task = loop.create_task(run_flow_task(flow_path, params))
    task.add_done_callback(lambda x: print(f"Task {x.result()} finished"))
    return task

In [ ]:
task = run_flow_async(
    os.path.join("test", "test_clustering.ipynb"),
    params={"traffic_percent": 10, "speed": "fast"},
)

In [ ]:
task.done()

In [ ]:
param_grid = {
    "traffic_percent": [1, 5, 10, 20, 50, 100],
    "speed": ["fast", "slow"],
    "workers": [1],
}

In [ ]:
# export


def iter_param_grid(param_grid):
    # https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/model_selection/_search.py
    for p in [param_grid]:
        # Always sort the keys of a dictionary, for reproducibility
        items = sorted(p.items())
        if not items:
            yield {}
        else:
            keys, values = zip(*items)
            for v in product(*values):
                params = dict(zip(keys, v))
                yield params

In [ ]:
assert [{"a": 1, "b": 1, "c": "hello"}, {"a": 2, "b": 1, "c": "hello"}] == list(
    iter_param_grid({"a": [1, 2], "b": [1], "c": ["hello"]})
)

In [ ]:
def sample_grid_space(param_grid: Dict[str, Iterable[Any]], num_samples: int):
    samples = []
    for i, sample in enumerate(iter_param_grid(param_grid)):
        samples.append(sample)
    if num_samples < len(samples):
        samples = pd.Series(samples).sample(num_samples).tolist()
    return samples

In [ ]:
sample_space = sample_grid_space({"a": [1, 2], "b": [1], "c": ["hello"]}, 1)
assert sample_space[0]["b"] == 1
assert sample_space[0]["c"] == "hello"
assert sample_space[0]["a"] == 1 or sample_space[0]["a"] == 2

In [ ]:
# export


def search_flow_grid(nb_path, param_grid, num_procs=None):
    max_process_count = int((multiprocessing.cpu_count() / 2) - 1)
    param_sample_space = sample_grid_space(param_grid, max_process_count)
    tasks = []
    for param_sample in param_sample_space:
        tasks.append(run_flow_async(nb_path, params=param_sample))
    return tasks

In [ ]:
# slow

nb_path = "/home/jovyan/git/sciflow/nbs/test/test_clustering.ipynb"
tasks = search_flow_grid(
    nb_path,
    {
        "traffic_percent": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50],
        "speed": ["fast"],
        "workers": [1],
    },
)

In [ ]:
# slow

[t.done() for t in tasks]

In [ ]:
# slow

[t.result() for t in tasks]

In [413]:
from sciflow.lake_experiment_loader import LakeExpLoader

In [414]:
LakeExpLoader("test_clustering").find_by_id(51)

ERROR IS: `ODBC error
state: HY000
native error code: 1040
message: [Dremio][Connector] (1040) Dremio failed to execute the query: select * from ra_s3.sciflow.experiments.test_clustering.runs where dir0 = 51
[30038]Query execution error. Details:[ 
VALIDATION ERROR: Table 'ra_s3.sciflow.experiments.test_clustering.runs' not found

SQL Query select * from ra_s3.sciflow.experiments.test_clustering.runs where dir0 = 51
startLine 1
startColumn 15
endLine 1
endColumn 19

[Error Id: 52780d00-7cf5-4464-af...[see log]`


DatabaseError: ODBC error
state: HY000
native error code: 1040
message: [Dremio][Connector] (1040) Dremio failed to execute the query: select * from ra_s3.sciflow.experiments.test_clustering.runs where dir0 = 51
[30038]Query execution error. Details:[ 
VALIDATION ERROR: Table 'ra_s3.sciflow.experiments.test_clustering.runs' not found

SQL Query select * from ra_s3.sciflow.experiments.test_clustering.runs where dir0 = 51
startLine 1
startColumn 15
endLine 1
endColumn 19

[Error Id: 52780d00-7cf5-4464-af...[see log]

In [ ]:
# export


@call_parse
def sciflow_check_flows():
    check_flows(Config())

In [ ]:
# export


@call_parse
def sciflow_run_flows():
    check_flows(Config(), "--no-pylint run")